In [ ]:
!pip install gymnasium

In [ ]:
import gymnasium as gym
import numpy as np
import time
from IPython import display



In [ ]:
env = gym.make('FrozenLake-v1', render_mode="ansi")


In [ ]:
env.P[0][3] # Transition model

In [ ]:
env.observation_space.n

In [ ]:
env.action_space.n

In [ ]:
def play(env, policy, render=False):
    state, _ = env.reset()
    total_reward = 0
    steps = 0
    done = False
    while not done:
        action = policy[state]
        next_state, reward, done, info, _ = env.step(action)
        total_reward += reward
        steps += 1
        if render:
            print(env.render())
            time.sleep(0.5)
            if not done:
                display.clear_output(wait=True)
        state = next_state

    return (total_reward, steps)

In [ ]:

policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
play(env, policy_0, True)


In [ ]:
policy_1 = np.asarray([0, 1, 1, 3, 1, 0, 2, 0, 1, 1, 2, 2, 3, 3, 1, 0])
play(env, policy_1, True)

In [ ]:
policy_2 = np.array([1, 1, 1, 3, 0, 1, 2, 3, 1, 1, 2, 3, 2, 2, 1, 3])
play(env, policy_2, True)

In [ ]:
policy_3 = np.array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0])
play(env, policy_3, True)

In [ ]:
def play_multiple_times(env, policy, max_episodes):
    success = 0
    list_of_steps = []
    for i in range(max_episodes):
        total_reward, steps = play(env, policy)

        if total_reward > 0:
            success += 1
            list_of_steps.append(steps)

    print(f'Number of successes: {success}/{max_episodes}')
    print(f'Average number of steps: {np.mean(list_of_steps)}')

In [ ]:
policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
play_multiple_times(env, policy_0, 1000)

In [ ]:
policy_1 = np.asarray([0, 1, 1, 3, 1, 0, 2, 0, 1, 1, 2, 2, 3, 3, 1, 0])
play_multiple_times(env, policy_1, 1000)

In [ ]:
policy_2 = np.array([1, 1, 1, 3, 0, 1, 2, 3, 1, 1, 2, 3, 2, 2, 1, 3])
play_multiple_times(env, policy_2, 1000)

In [ ]:
policy_3 = np.array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0])
play_multiple_times(env, policy_3, 1000)

In [ ]:
def policy_evaluation(env, policy, max_iters=500, gamma=0.9):
    # Initialize the values of all states to be 0
    v_values = np.zeros(env.observation_space.n)

    for i in range(max_iters):
        prev_v_values = np.copy(v_values)

        # Update the value of each state
        for state in range(env.observation_space.n):
            action = policy[state]

            # Compute the q-value of the action
            q_value = 0
            for prob, next_state, reward, done in env.P[state][action]:
                q_value += prob * (reward + gamma * prev_v_values[next_state])

            v_values[state] = q_value # update v-value

        # Check convergence
        if np.all(np.isclose(v_values, prev_v_values)):
            print(f'Converged at {i}-th iteration.')
            break

    return v_values

In [ ]:
policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
v_values_0 = policy_evaluation(env, policy_0)
print(v_values_0)

In [ ]:
# policy_1 = np.asarray([0, 1, 1, 3, 1, 0, 2, 0, 1, 1, 2, 2, 3, 3, 1, 0])
# v_values_1 = policy_evaluation(env, policy_1)
# print(v_values_1)

In [ ]:
# np.all(v_values_1 >= v_values_0)

In [ ]:
policy_2 = np.array([1, 1, 1, 3, 0, 1, 2, 3, 1, 1, 2, 3, 2, 2, 1, 3])
v_values_2 = policy_evaluation(env, policy_2)
print(v_values_2)

In [ ]:
np.all(v_values_2 >= v_values_1)

In [ ]:
policy_3 = np.array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0])
v_values_3 = policy_evaluation(env, policy_3)
print(v_values_3)

In [ ]:
np.all(v_values_3 >= v_values_2)

In [ ]:
def value_iteration(env, max_iters=500, gamma=0.9):
    # initialize
    v_values = np.zeros(env.observation_space.n)

    for i in range(max_iters):
        prev_v_values = np.copy(v_values)

        # update the v-value for each state
        for state in range(env.observation_space.n):
            q_values = []

            # compute the q-value for each action that we can perform at the state
            for action in range(env.action_space.n):
                q_value = 0
                # loop through each possible outcome
                for prob, next_state, reward, done in env.P[state][action]:
                    q_value += prob * (reward + gamma * prev_v_values[next_state])

                q_values.append(q_value)

            # select the max q-values
            best_action = np.argmax(q_values)
            v_values[state] = q_values[best_action]

        # check convergence
        if np.all(np.isclose(v_values, prev_v_values)):
            print(f'Converged at {i}-th iteration.')
            break

    return v_values

In [ ]:
optimal_v_values = value_iteration(env, max_iters=500, gamma=0.9)

In [ ]:
optimal_v_values

In [ ]:
def policy_extraction(env, v_values, gamma=0.9):
    # initialize
    policy = np.zeros(env.observation_space.n, dtype=np.int32)

    # loop through each state in the environment
    for state in range(env.observation_space.n):
        q_values = []
        # loop through each action
        for action in range(env.action_space.n):
            q_value = 0
            # loop each possible outcome
            for prob, next_state, reward, done in env.P[state][action]:
                q_value += prob * (reward + gamma * v_values[next_state])

            q_values.append(q_value)

        # select the best action
        best_action = np.argmax(q_values)
        policy[state] = best_action

    return policy

In [ ]:
optimal_policy = policy_extraction(env, optimal_v_values, gamma=0.9)

In [ ]:
optimal_policy

In [ ]:
play(env, optimal_policy, False)

In [ ]:
play_multiple_times(env, optimal_policy, 1000)

In [ ]:
def policy_iteration(env, max_iters=500, gamma=0.9):
    #ini
    policy = np.zeros(env.observation_space.n)
    v_values = np.zeros(env.observation_space.n)

    for i in range(max_iters):
        v_values = policy_evaluation(env, policy, max_iters, gamma)
        prev_policy = policy
        policy = policy_extraction(env, v_values, gamma)

        if np.all(np.isclose(policy, prev_policy)):
            print(f"Policy iteration converged at {i}-th iteration.")
            break
    
    return policy


In [ ]:
iterated_policy = policy_iteration(env, 500, 0.9)


In [ ]:
print(optimal_policy)
print(iterated_policy)

In [ ]:
play_multiple_times(env, optimal_policy, 1000)
play_multiple_times(env, iterated_policy, 1000)

# Nhận xét
Cả hai thuật toán đều là các cách để tìm ra chiến lược tối ưu cho các bài toán MDP. Tiếp tục qua một bước Policy Extraction, Value Iteration có thể tìm ra chiến lược tối ưu (optimal policy) giống với kết quả của Policy Iteration. Về bản chất, hai thuật toán này đều gần như giống nhau.

Với PI, thuật toán sẽ hội tụ sau số lần lặp ít hơn VI, những vòng lặp cuối của VI thường không thay đổi các giá trị nhiều, mặc dù chiến lược tối ưu đã có thể xác định từ nhiều lần lặp trước.

